In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator().build(number, verbose=False)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count()*5, number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9931211082570558, 0.9921901485360478)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9993009481087198, 0.9992271011154659)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe()

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
0,ItalianFiscalCode,ItalianVAT,CadastreCode,Error,NaN,NaN,ItalianZIPCode,NaN,Region,Municipality,...,CountryCode,Name,Surname,String,EMail,Error,NaN,BiologicalSex,Boolean,NumericId
1,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,NaN,String,EMail,NaN,Date,BiologicalSex,NaN,NaN
2,ItalianFiscalCode,NaN,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,NaN,Surname,String,EMail,NaN,Error,BiologicalSex,Boolean,NumericId
3,NaN,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,Region,Municipality,...,NaN,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NumericId
4,ItalianFiscalCode,NaN,CadastreCode,Document,NaN,Address,ItalianZIPCode,ProvinceCode,NaN,NaN,...,NaN,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean,NumericId
5,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Plate,NaN,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,NaN,Error,PhoneNumber,Date,BiologicalSex,NaN,NumericId
6,NaN,ItalianVAT,CadastreCode,Document,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,NaN,NaN,String,EMail,PhoneNumber,Date,NaN,Boolean,NaN
7,ItalianFiscalCode,ItalianVAT,NaN,NaN,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean,Error
8,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Address,NaN,ProvinceCode,Region,Municipality,...,CountryCode,NaN,Surname,NaN,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,NumericId
9,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,CountryCode,Name,Surname,String,NaN,PhoneNumber,NaN,NaN,Boolean,NumericId


In [10]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Country', 'Error'): 182,
         ('Error', 'ItalianFiscalCode'): 27,
         ('Error', 'Country'): 207,
         ('Error', 'NumericId'): 43,
         ('ItalianFiscalCode', 'Error'): 18,
         ('Error', 'String'): 42,
         ('Country', 'String'): 98,
         ('String', 'Address'): 1010,
         ('NumericId', 'Integer'): 1,
         ('Country', 'Municipality'): 355,
         ('Error', 'Document'): 114,
         ('NaN', 'CountryCode'): 66,
         ('CountryCode', 'NaN'): 67,
         ('Address', 'String'): 1153,
         ('Error', 'CountryCode'): 40,
         ('String', 'Error'): 75,
         ('Error', 'Surname'): 1,
         ('Name', 'Country'): 5,
         ('String', 'Country'): 232,
         ('Address', 'Municipality'): 16,
         ('Error', 'Municipality'): 13,
         ('ItalianZIPCode', 'Error'): 11,
         ('String', 'Municipality'): 104,
         ('Error', 'ProvinceCode'): 22,
         ('Error', 'ItalianVAT'): 15,
         ('CountryCode', 'Error'): 81,
   